# Divergent data reduction for Amor

In this notebook, we will look at the reduction workflow for reflectometry data collected from the PSI
[Amor](https://www.psi.ch/en/sinq/amor) instrument in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).
This is a living document and there are plans to update this as necessary with changes in the data reduction methodology and code.

We will begin by importing the modules that are necessary for this notebook.

In [ ]:
import scipp as sc
from ess import amor
import ess

In [ ]:
logger = ess.logging.configure_workflow('amor_reduction',
                                        filename='amor.log')

## The Amor beamline

Before we can load the data, we need to define the parameters of the beamline and briefly discuss the measurement philosophy.
We begin by defining the convention for naming angles in our set-up.
We use the Fig. 5 from the paper by [Stahn & Glavic (2016)](#stahn2016),
which is reproduced below (along with its caption).

![Figure5](https://ars.els-cdn.com/content/image/1-s2.0-S0168900216300250-gr5.jpg)

The yellow area shows the incoming and reflected beam, both with the divergence $\Delta \theta$.
The inclination of the sample relative to the centre of the incoming beam (here identical to the instrument horizon) is called $\omega$, and the respective angle of the reflected beam relative to the same axis is $\gamma$.

In general the detector centre is located at $\gamma_{\rm D} = 2\omega$.
These are instrument coordinates and should not be confused with the situation on the sample, where the take-off angle of an individual neutron trajectory is called $\theta$.

### The supermirror reference

The normalisation of data from the Amor instrument in divergent mode requires a reference measurement of a neutron supermirror.
The supermirror is not a perfect supermirror, and is described with some properties, an $m$-value, a critical edge, and an $\alpha$, from which we can calibrate the supermirror. 
This reference measurement facilitates two normalisations on our data:
- normalisation of neutron count per unit time, assuming that the instrument flux is constant between the supermirror measurement and our sample measurement,
- normalisation over the detector pixels, to account for differences in pixel efficiency.
It is important when this normalisation is performed that the differences in count time and beam footprint are accounted for such that the measurements are commensurate.

The `amor` module provides a helper function that generates the default beamline parameters.
This function requires the sample rotation angle ($\omega$) as an input to fully define the beamline.
In the future, all these beamline parameters (including the sample rotation) will be included in the file meta data.
For now, we must define this manually, and the rotation is different for the sample and reference files.

In [ ]:
sample_rotation = sc.scalar(0.7989, unit='deg')
sample_beamline = amor.make_beamline(sample_rotation=sample_rotation)
reference_rotation = sc.scalar(0.7589, unit='deg')
reference_beamline = amor.make_beamline(sample_rotation=reference_rotation)

## Loading the data

The `sample.nxs` file is the experimental data file of interest,
while `reference.nxs` is the reference measurement of the neutron supermirror.
The `amor.load` function can be used to load these files and perform some early preprocessing:

- The `tof` values are converted from nanoseconds to microseconds.
- The raw data contains events coming from two pulses, and these get folded into a single `tof` range.

We show and plot the resulting `scipp.DataArray` for just the `sample` below. 

In [ ]:
sample = amor.load(amor.data.get_path("sample.nxs"),
                   beamline=sample_beamline)
reference = amor.load(amor.data.get_path("reference.nxs"),
                      beamline=reference_beamline)
sample

By simply plotting the data, we get a first glimpse into the data contents.

In [ ]:
sc.plot(sample)

### Correcting the position of the detector pixels

**Note:** once new Nexus files are produced, this step should go away. 

The pixel positions are wrong in the `sample.nxs` and `reference.nxs` files, and require an ad-hoc correction.
We apply an arbitrary shift in the vertical (`y`) direction.
We first move the pixels down by 0.955 degrees,
so that the centre of the beam goes through the centre of the top half of the detector blades
(the bottom half of the detectors was turned off).
Next, we move all the pixels so that the centre of the top half of the detector pixels lies at an angle of $2 \omega$,
as described in the beamline diagram.

In [ ]:
logger.info("Correcting pixel positions in 'sample.nxs'")
def pixel_position_correction(data: sc.DataArray):
    return data.coords['position'].fields.z * sc.tan(2.0 *
                                                     data.coords['sample_rotation'] -
                                                     (0.955 * sc.units.deg))
sample.coords['position'].fields.y += pixel_position_correction(sample)
reference.coords['position'].fields.y += pixel_position_correction(reference)

We now check that the detector pixels are in the correct position by showing the instrument view

In [ ]:
amor.instrument_view(sample)

## Coordinate transformation graph

To compute the wavelength $\lambda$, the scattering angle $\theta$, and the $Q$ vector for our data,
we construct a coordinate transformation graph.

It is based on classical conversions from `tof` and pixel `position` to $\lambda$ (`wavelength`),
$\theta$ (`theta`) and $Q$ (`Q`),
but comprises a number of modifications.

The computation of the scattering angle $\theta$ includes a correction for the Earth's gravitational field which bends the flight path of the neutrons.
The angle can be found using the following expression

$$\theta = \sin^{-1}\left(\frac{\left\lvert y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2} \right\rvert }{L_{2}}\right) - \omega$$

where $m_{\rm n}$ is the neutron mass,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant.

For a graphical representation of the above expression,
we consider once again the situation with a convergent beam onto an inclined sample.

![specular_reflection](amor_specular_reflection.png)

The detector (in green), whose center is located at an angle $\gamma_{\rm D}$ from the horizontal plane,
has a physical extent and is measuring counts at multiple scattering angles at the same time.
We consider two possible paths for neutrons.
The first path (cyan) is travelling horizontally from the source to the sample and subsequently,
following specular reflection, hits the detector at $\gamma_{\rm D}$ from the horizontal plane.
From the symmetry of Bragg's law, the scattering angle for this path is $\theta_{1} = \gamma_{\rm D} - \omega$.

The second path (red) is hitting the bottom edge of the detector.
Assuming that all reflections are specular,
the only way the detector can record neutron events at this location is if the neutron originated from the bottom part of the convergent beam.
Using the same symmetry argument as above, the scattering angle is $\theta_{2} = \gamma_{2} - \omega$. 

This expression differs slightly from the equation found in the computation of the $\theta$ angle in other techniques such as
[SANS](https://docs.mantidproject.org/nightly/algorithms/Q1D-v2.html#q-unit-conversion),
in that the horizontal $x$ term is absent,
because we assume a planar symmetry and only consider the vertical $y$ component of the displacement.

The conversion graph is defined in the reflectometry module,
and can be obtained via

In [ ]:
graph = amor.conversions.specular_reflection()
sc.show_graph(graph, simplified=True)

## Computing the wavelength

To compute the wavelength of the neutrons,
we request the `wavelength` coordinate from the `transform_coords` method by supplying our graph defined above
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`).

We also exclude all neutrons with a wavelength lower than 2.4 &#8491;.

In [ ]:
sample_wav = sample.transform_coords(["wavelength"], graph=graph)
wavelength_edges = sc.array(dims=['wavelength'], values=[2.4, 16.0], unit='angstrom')
sample_wav = sc.bin(sample_wav, edges=[wavelength_edges])
sample_wav

In [ ]:
sample_wav.bins.concatenate('detector_id').plot()

In [ ]:
reference_wav = reference.transform_coords(["wavelength"], graph=graph)
reference_wav = sc.bin(reference_wav, edges=[wavelength_edges])

## Compute the angle and perform the footprint correction

Using the same method, we can compute the angle of reflectance ($\theta$) and therefore correct for the footprint of the beam. 

In [ ]:
sample_theta = sample_wav.transform_coords(['theta'], graph=graph)

The theta coordinate is a bin-edge coordinate (with only two edges), so we will find the centres of these. 

In [ ]:
theta_centres = sc.to_unit(sc.midpoints(sample_theta.coords['theta'], 'theta'), 'deg')

From these centres, we can calculate the footprint of the beam on the sample.

In [ ]:
beam_on_sample = sample_theta.coords['beam_size'] / sc.sin(theta_centres)

Then we can determine the footprint scaling factor for the given theta angle as follows. 

In [ ]:
fwhm_to_std = sc.scalar(2.) * sc.sqrt(sc.scalar(2.) * sc.log(sc.scalar(2.)))
footprint_scale = sc.erf(sample_theta.coords['sample_size'] / beam_on_sample * fwhm_to_std)
sample_theta.coords['footprint_scale'] = footprint_scale.squeeze()

Then we repeat this process for the reference.

In [ ]:
reference_theta = reference_wav.transform_coords(['theta'], graph=graph)
theta_centres = sc.to_unit(sc.midpoints(reference_theta.coords['theta'], 'theta'), 'deg')
beam_on_sample = reference_theta.coords['beam_size'] / sc.sin(theta_centres)
fwhm_to_std = sc.scalar(2.) * sc.sqrt(sc.scalar(2.) * sc.log(sc.scalar(2.)))
footprint_scale = sc.erf(reference_theta.coords['sample_size'] / beam_on_sample * fwhm_to_std)
reference_theta.coords['footprint_scale'] = footprint_scale.squeeze()

## Compute the Q vector

Once again using the same method, we can compute the $Q$ vector,
which now depends on both detector position (id) and wavelength

In [ ]:
sample_q = sample_theta.transform_coords(["Q"], graph=graph)
reference_q = reference_theta.transform_coords(["Q"], graph=graph)
q_edges = sc.geomspace(dim='Q', start=0.008, stop=0.08, num=201, unit='1/angstrom')
sample_q_binned = sc.bin(sample_q, edges=[q_edges])
reference_q_binned = sc.bin(reference_q, edges=[q_edges])
sc.plot({'sample': sample_q_binned.sum('detector_id'), 'uncalibrated reference': reference_q_binned.sum('detector_id')}, norm="log")

## Calibration of the super-mirror

In order to normalise the data to give reflectivity data, as mentioned above, we use a measurement from a neutron super-mirror. 
However, first we must calibrate the super-mirror measurement. 
The calibration of the super-mirror depends on the properties of the super-mirror, and follows the equation below, 

$$ n(q) = 
  \begin{cases}
    1, & \text{where } q < c_{\mathrm{sm}} \\
    [1-\alpha(q - c_{\mathrm{sm}})]^{-1}, & \text{where } c_{\mathrm{sm}} \leq q \leq mc_{\mathrm{sm}} \\
    0, & \text{where } q > mc_{\mathrm{sm}},
  \end{cases}  
$$

where $\alpha$, $m$, and $c_{\mathrm{sm}}$ are super-mirror properties. 

The number of counts in each of the detector/$Q$ bins are then summed and the calibration factor is found and the two are divided. 

In [ ]:
reference_q_summed = reference_q_binned.bins.sum()
calibration_factor = amor.calibrations.supermirror_calibration(reference_q_summed.coords['Q'])
reference_q_summed_cal = reference_q_summed * calibration_factor

The effect on the reference measurement can be seen in the plot below. 

In [ ]:
sc.plot({'Uncalibrated': reference_q_summed.sum('detector_id'), 
         'Calibrated': reference_q_summed_cal.sum('detector_id')}, 
        norm='log')

## Normalization by the super-mirror

For each of the measurements, we should determine the number of counts in each bins and normalise this by the total number of counts in the measurement and the footprint scale factor.
This footprint scale factor accounts for the fact that the illuminated area of the sample depends on the angle of incidence (which as we noted previously are different for the sample and the reference).

In [ ]:
sample_q_summed = sample_q_binned.bins.sum()

sample_ncounts = sample_q_summed.sum()
sample_norm = sample_q_summed / (sample_ncounts * reference_q.coords['footprint_scale'])
reference_ncounts = reference_q_summed_cal.sum()
reference_norm = reference_q_summed_cal / (reference_ncounts)

Now, we should obtain the final normalised data by dividing the two datasets. 

In [ ]:
ratio = sample_norm.coords['footprint_scale'] / reference_norm.coords['footprint_scale']
del sample_norm.coords['footprint_scale']
del reference_norm.coords['footprint_scale']

In [ ]:
normalized = sample_norm / reference_norm
normalized.coords['footprint_ratio'] = ratio
sc.plot(normalized)

The above plot shows the two-dimensional plots of $Q$ and the detector pixel number.
Here, we note that there are a large number of pixels, where there was no neutrons detected in the reference measurements, leading to values of `nan` and `inf` in the normalised data.
Therefore, we should mask these pixels before finding the mean along the `'detector_id'` dimension. 

In [ ]:
normalized.masks['no_reference_neutrons'] = (reference_norm == sc.scalar(0)).data

Finally, we perfrom a mean along the `'detector_id'` to obtain the normalised reflectometry profile.
The mean is used to account for the fact that some pixels are masked for particular $Q$-bins and this will ensure this is accounted for appropriately.

In [ ]:
sc.plot(normalized.mean('detector_id'), norm='log')

## Make a $(\lambda, \theta)$ map

A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins.
To achieve this, we request two output coordinates from the `transform_coords` method.

In [ ]:
sample_theta = sample.transform_coords(["theta", "wavelength"], graph=graph)

Then, we concatenate all the events in the `detector_id` dimension

In [ ]:
sample_theta = sample_theta.bins.concatenate('detector_id')

Finally, we bin into the existing `theta` dimension, and into a new `wavelength` dimension,
to create a 2D output

In [ ]:
nbins = 165
theta_edges = sc.linspace(dim='theta', start=0.0, stop=1.2, num=nbins, unit='deg')
wavelength_edges = sc.linspace(dim='wavelength', start=0, stop=15.0, num=nbins, unit='angstrom')
binned = sc.bin(sample_theta, edges=[sc.to_unit(theta_edges, 'rad'), wavelength_edges])
binned

In [ ]:
binned.bins.sum().plot()

This plot can be used to check if the value of the sample rotation angle $\omega$ is correct.
The bright triangles should be pointing back to the origin $\lambda = \theta = 0$.

## References

<div id='stahn2016'></div>
Stahn J., Glavic A., **2016**,
*Focusing neutron reflectometry: Implementation and experience on the TOF-reflectometer Amor*,
[Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment, 821, 44-54](https://doi.org/10.1016/j.nima.2016.03.007)